In [23]:
import pandas as pd
import numpy as np
import json

In [79]:
codes_df = pd.read_csv('by_codes.out', header=None, sep=',')
codes_df.drop(0, inplace=True)
codes_df[0] = codes_df[0].map(str)
codes_df[2] = codes_df[2].map(str)
codes_df.head()

,0,1,2,3
1,102,'HOMICIDE-NEGLIGENT-VEHICLE',121,15
2,102,'HOMICIDE-NEGLIGENT-VEHICLE',122,78
3,103,'HOMICIDE-NEGLIGENT UNCLASSIFIE',125,33
4,104,'RAPE',153,1821
5,104,'RAPE',155,1605


In [80]:
#For each KY_CD, list all associated PD_CD
ky_to_pd = codes_df.groupby(0).agg({2: lambda x: ','.join(x)})
ky_to_pd[2] = ky_to_pd[2].map(lambda x: x.split(','))
ky_to_pd.head()

,2
0,
102,"[121, 122]"
103,[125]
104,"[153, 155, 157, 159]"
105,"[419, 360, 361, 363, 365, 366, 367, 371, 373, ..."
106,"[101, 186, 321, 639, 749, 759, 779, 105, 106, ..."


In [81]:
#For each PD_CD, list all associated KY_CD
pd_to_ky = codes_df.groupby(2).agg({0: lambda x: ','.join(x)})
pd_to_ky[0] = pd_to_ky[0].map(lambda x: x.split(','))
pd_to_ky.head()

,0
2,
101,"[106, 344, 359, 578]"
104,[126]
105,[106]
106,[106]
107,[126]


In [82]:
#Check if a PC_CD has 2+ KY_CDs it is associated with
mask = [False]*len(pd_to_ky)
for i in range(len(mask)):
    if len(pd_to_ky[0][i]) > 1:
        mask[i] = True
pd_to_ky[mask].head()

,0
2,
101,"[106, 344, 359, 578]"
109,"[106, 344]"
113,"[126, 344, 578]"
117,"[116, 126, 355]"
157,"[104, 233]"


In [83]:
#drop any KY_CD-PC_CD combinations where the PC_CD is associated with 2+ KY_CDs
#and less than 5 crimes used that KY_CD-PC_CD combination
drop_list = []
for ids in pd_to_ky[mask].index.values:
    mask1 = codes_df[2] == ids
    mask2 = codes_df[3] < 5
    mask3 = mask1&mask2
    drop_list.extend(codes_df[mask3].index.values)

for i in range(len(drop_list)):
    codes_df = codes_df.drop(drop_list[i])
codes_df.iloc[0:10]

,0,1,2,3
1,102,'HOMICIDE-NEGLIGENT-VEHICLE',121,15
2,102,'HOMICIDE-NEGLIGENT-VEHICLE',122,78
3,103,'HOMICIDE-NEGLIGENT UNCLASSIFIE',125,33
4,104,'RAPE',153,1821
5,104,'RAPE',155,1605
6,104,'RAPE',157,8939
7,104,'RAPE',159,1426
9,105,'ROBBERY',360,396
10,105,'ROBBERY',361,3104
11,105,'ROBBERY',363,333


In [84]:
#Run again now that we have removed outliers
ky_to_pd = codes_df.groupby(0).agg({2: lambda x: ','.join(x)})
ky_to_pd[2] = ky_to_pd[2].map(lambda x: x.split(','))

pd_to_ky = codes_df.groupby(2).agg({0: lambda x: ','.join(x)})
pd_to_ky[0] = pd_to_ky[0].map(lambda x: x.split(','))


In [89]:
#Save as dictionaries
ky_to_pd_dict = ky_to_pd.to_dict()
ky_to_pd_dict = ky_to_pd_dict[2]
pd_to_ky_dict = pd_to_ky.to_dict()
pd_to_ky_dict = pd_to_ky_dict[0]

In [86]:
#Save dictionaries
json.dump(pd_to_ky_dict, open("pd_to_ky_dict.txt",'w'))
json.dump(ky_to_pd_dict, open("ky_to_pd_dict.txt",'w'))

In [92]:
#Check expected number of valid KY_CDs
valid_count = 0
for entry in codes_df.index:
    if codes_df[0][entry] in pd_to_ky_dict[codes_df[2][entry]]:
        valid_count = valid_count + codes_df[3][entry]
valid_count

5096515

In [93]:
#Check expected number of valid PD_CDs
valid_count = 0
for entry in codes_df.index:
    if codes_df[2][entry] in ky_to_pd_dict[codes_df[0][entry]]:
        valid_count = valid_count + codes_df[3][entry]
valid_count

5096515